<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="600" height="500"></center>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf)!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## `client`
| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `text` | Whether the client's credit is in default |
| `housing` | `text` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `text` | Whether the client has an existing personal loan |

## `campaign`
| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `integer` | Campaign ID |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `text` | Outcome of the previous campaign |
| `campaign_outcome` | `integer` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

## `economics`
| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

- Split the data into three, one with information about the client, another containing campaign data, and a third to store information about economics at the time of the campaign.
- Rename columns, including "client_id" to "id" in client (leave as-is in the other subsets); "duration" to "contact_duration", "previous" to "previous_campaign_contacts", "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in campaign; and "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in economics.
- Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values.
- Remove periods from the "job" column.
- Convert "previous_outcome" and "campaign_outcome" values to binary (1 or 0), including the conversion of "nonexistent" to 0.
- Add a campaign_id as the first column in campaign, where all rows have a value of 1.
- Create a datetime column called last_contact_date, in the format of "month-day-year", where the year is 2022, and the month and day values are taken from the "month" and "day_of_week" columns.
- Remove any redundant data that might have been used to create new columns, ensuring the columns in each subset of the data match the table displayed in the notebook.
- Save the three subsets to csv files without an index.
- Create a dictionary called database_design using the three table names as keys and assigning values as the respective subset DataFrames you have created.
- Print database_design to check your solution.

In [126]:
# Start coding...
import pandas as pd
import numpy as np

In [127]:
bank_marketing = pd.read_csv('bank_marketing.csv')

In [128]:
bank_marketing.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [129]:
bank_marketing.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

# Creating client table

In [130]:
# need id, age, job, marital status, education, credit_default, housing, loan
client = bank_marketing.loc[:, ['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan',]]
client.head()

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [131]:
client.rename(columns={'client_id':'id'}, inplace=True)
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


# Creating campaign table

In [132]:
bank_marketing.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [133]:
campaign = bank_marketing.loc[:,['client_id', 'campaign', 'month', 'day_of_week', 
                                 'duration', 'pdays', 'previous', 'poutcome', 'y']]
campaign.head()

,client_id,campaign,month,day_of_week,duration,pdays,previous,poutcome,y
0,0,1,may,mon,261,999,0,nonexistent,no
1,1,1,may,mon,149,999,0,nonexistent,no
2,2,1,may,mon,226,999,0,nonexistent,no
3,3,1,may,mon,151,999,0,nonexistent,no
4,4,1,may,mon,307,999,0,nonexistent,no


In [134]:
# dict of columns to be renamed
campaign_rename = {
    'duration': 'contact_duration',
    'previous': 'previous_campaign_contacts',
    'y': 'campaign_outcome',
    'poutcome': 'previous_outcome',
    'campaign': 'number_contacts'
}

# rename columns
campaign.rename(columns=campaign_rename, inplace=True)
campaign.head()

,client_id,number_contacts,month,day_of_week,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,mon,261,999,0,nonexistent,no
1,1,1,may,mon,149,999,0,nonexistent,no
2,2,1,may,mon,226,999,0,nonexistent,no
3,3,1,may,mon,151,999,0,nonexistent,no
4,4,1,may,mon,307,999,0,nonexistent,no


# Creating the economics table

In [135]:
# slice economics table
economics = bank_marketing.loc[:, ['client_id', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed']]
economics.head()

,client_id,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,0,1.1,93.994,-36.4,4.857,5191.0
1,1,1.1,93.994,-36.4,4.857,5191.0
2,2,1.1,93.994,-36.4,4.857,5191.0
3,3,1.1,93.994,-36.4,4.857,5191.0
4,4,1.1,93.994,-36.4,4.857,5191.0


In [136]:
# economics columns to be renamed
economics_rename = {
    'euribor3m': 'euribor_three_months',
    'nr_employed': 'number_employed'
}
# rename columns
economics.rename(columns=economics_rename, inplace=True)
economics.head()

,client_id,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_three_months,number_employed
0,0,1.1,93.994,-36.4,4.857,5191.0
1,1,1.1,93.994,-36.4,4.857,5191.0
2,2,1.1,93.994,-36.4,4.857,5191.0
3,3,1.1,93.994,-36.4,4.857,5191.0
4,4,1.1,93.994,-36.4,4.857,5191.0


# Data Cleaning

In [137]:
# education col
# replace '.' with '-' and 'unknown' with null values
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

# job col
# remove '.'
client["job"] = client["job"].str.replace(".", "")

In [138]:
# previous_outcome col
# convert to binary
outcome_mapping = {
    'failure': 0,
    'nonexistent': 0,
    'success': 1
}

campaign['previous_outcome'] = campaign['previous_outcome'].replace(outcome_mapping)

In [139]:
campaign.head()

,client_id,number_contacts,month,day_of_week,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,mon,261,999,0,0,no
1,1,1,may,mon,149,999,0,0,no
2,2,1,may,mon,226,999,0,0,no
3,3,1,may,mon,151,999,0,0,no
4,4,1,may,mon,307,999,0,0,no


In [140]:
# add new column campaign_id with all rows having value of 1
campaign = campaign.assign(campaign_id=1)

# Move the 'campaign_id' column to the first position
campaign = campaign.reindex(['campaign_id'] + list(campaign.columns[:-1]), axis=1)

campaign.head()

,campaign_id,client_id,number_contacts,month,day_of_week,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,1,0,1,may,mon,261,999,0,0,no
1,1,1,1,may,mon,149,999,0,0,no
2,1,2,1,may,mon,226,999,0,0,no
3,1,3,1,may,mon,151,999,0,0,no
4,1,4,1,may,mon,307,999,0,0,no


In [141]:
# Create a new datetime column called 'last_contact_date'
campaign['month'] = campaign['month'].str.capitalize()
campaign['year'] = "2022"
campaign['day_of_week'] = campaign['day_of_week'].str.capitalize()

campaign['last_contact_date'] = campaign['year'] + '-' + campaign['month'] + '-' + campaign['day_of_week']

campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'],
                                              format='%Y-%b-%a')

# drop month year day of week cols
campaign.drop(columns=['month',
                      'year',
                      'day_of_week'], 
             inplace=True)

In [142]:
## Structuring the dictionary
database_design = {"client": client,
                   "campaign": campaign,
                   "economics": economics}
print(database_design)

{'client':           id  age          job  ... credit_default housing loan
0          0   56    housemaid  ...             no      no   no
1          1   57     services  ...        unknown      no   no
2          2   37     services  ...             no     yes   no
3          3   40        admin  ...             no      no   no
4          4   56     services  ...             no      no  yes
...      ...  ...          ...  ...            ...     ...  ...
41183  41183   73      retired  ...             no     yes   no
41184  41184   46  blue-collar  ...             no      no   no
41185  41185   56      retired  ...             no     yes   no
41186  41186   44   technician  ...             no      no   no
41187  41187   74      retired  ...             no     yes   no

[41188 rows x 8 columns], 'campaign':        campaign_id  client_id  ...  campaign_outcome  last_contact_date
0                1          0  ...                no         2022-05-01
1                1          1  ...    

In [143]:
# Saving the data
## Store the DataFrames
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)